In [ ]:
from IPython.core.display import HTML

def css_styling():
    styles = open("../Data/www/styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [ ]:
import glob
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from pylab import imread
from random import randint
from scipy.stats import norm

In [ ]:
%matplotlib inline

# Project

For this project, we will download images of cells on plates from [Cell Image Library](http://www.cellimagelibrary.org/pages/project_20269) and write code to identify the contour of the cells, and then measure how well our code performs.


In [ ]:
# Write your code here to load all images (and maybe visualize them...)

paths = glob.glob('../Data/Day8-Image-Analysis/Cell_images/BBBC022_v1_images_20585w1/*')


In [ ]:
# Write code to check some of the properties of the image(s) you loaded


In [ ]:
# Write some code to focus on the important parts of the histogram of intensities


We need to think about what is `background` and what is `foreground`. A possible solution is to define a threshold and set all values below (background) to zero and all above (foreground) equal to 1...

In [ ]:
# Write code here to explore use of thresholds to differenciate background from foreground

In [ ]:
fig, plots = plt.subplots(figsize=(8, 25), nrows=10, ncols=2)

for i in range(10):
    plots[i][0].imshow(images[i], cmap=cm.Greys)
    plots[i][1].imshow(masks[i][:,:], cmap=cm.Greys)
    
plt.tight_layout()
plt.show()

Because of the close proximity of the cells in some regions, likely there is no threshold single threshold that neatly separates all the cells. But, maybe you already have a good enough threshold that appears to separate most of the cells.  Then what you need to do it actually identify/mark the individual cells.  

## Creating a validation image

The best way to test your code is to create a (set of) image(s) that countain "cells" created by you.  You would specify distributions in intensities for foreground, background, position of center of cells, and shape of cells. At first, it might be best to generate validation sets that are relatively easy!

What is easy? backgroung is 0, foreground is 1000, cells are circles and there are only a handful of cells in the image.

In [ ]:
# write your code for generating validation images here

def create_validation_image(number_cells = 1, cell_radius = 5, Lx = 100, Ly = 100, 
                            mean_backgd = 0, stdev_bacgdk = 0, mean_foregd = 1000, stdev_foregd = 0):
    """
    """
    
    return image, cell_centers

Now you can use the easy images you created to develop your code for detecting cells

In [ ]:
# write your code here
# We are going to assign different labels to each cluster of foreground pixels.
# But which pixels can potentially belong to the same cluster? I suggest using the eight 
# pixels around each center pixel.


def get_neighbors(x, y, Lx, Ly):
    """
    Takes coordinates x and y and returns list of tuples with coordinates of 8 neighbors of (x,y)
    
    input:
        x - int
        y - int
        Lx - int
        Ly - int
        
    output:
        neighbors - list of tuples
    """

    return neighbors


cells = {}


If you set things up the same way as I, cells is a dictionary with cell labels as key and values that are a list of tuples with the coordinates of the pixels belonging to the cells.

To calculate the area of a cell, we just need to count the number of pixels in each list.

Retrieving the center of mass coordinates is also straightforward. 

In [ ]:
print(cell_centers)
print()
print(cells.keys())
for label in cells.keys():
    print(label, len(cells[label]), get_center_of_mass(cells[label]))
    

In [ ]:
# Write your code here

def get_center_of_mass(pixels):
    """
    """
    
    return xcm, ycm